# Querying MPC for objects in Allen Fields

## First load the pointing group info

In [1]:
import numpy as np
import pickle
from astropy.io.votable import parse
import matplotlib.pyplot as plt
import pandas as pd

import healpy as hp
import pointing_groups as pg

import requests
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs import NoConvergence

from queryMPC import runMPCRequests
from queryMPC import matchPixels

%matplotlib inline

In [2]:
with open('PickledPointings.pkl', 'rb') as f:
    Pointing_Groups = pickle.load(f)
    
den_visit,den_ccd,den_stellarnum = np.loadtxt('ref_data.txt',unpack=True)
unique_den_visit = np.unique(den_visit)
max_stellarnum = []
_= [ max_stellarnum.append(np.max(den_stellarnum[den_visit==visit])) for visit in unique_den_visit ]
max_stellarnum = np.array(max_stellarnum)

for i,group in enumerate(Pointing_Groups):
    group_max = np.concatenate([ (max_stellarnum[(unique_den_visit==x)]) for x in np.array(Pointing_Groups[i]["visit_id"],dtype=np.float_) ])
    if not(group_max.size):
        group_max=[-9999.]
    group_max = np.max(group_max)
    Pointing_Groups[i]['stellar_density'] = group_max
    
# visitToIndex is a dictionary that returns the index of a given visit
visitToIndex = {}
for i,pointing in enumerate(Pointing_Groups):
    for visit in pointing['visit_id']:
        visitToIndex[str(visit)] = i

## Set up tools for MPC querying

## Pick a pointing group and get data with `runMPCRequests`

This will query the online MPC database and get a dataframe with all MPC objects within 90 arcminutes of the center of each pointing.

In [3]:
pgNum=300 # Pointing Group Number as assigned in NEO_Fields
# Get all MPC objects within 90 arcminutes of the pointing group center
resultsDF = runMPCRequests(Pointing_Groups[pgNum], str(pgNum))
# Initialize ccd, x_pixel, and y_pixel values
# This prepares the dataframe for the pixel matching
resultsDF['ccd'] = -99
resultsDF['x_pixel'] = -99
resultsDF['y_pixel'] = -99
# Calculate total motion
resultsDF['total_motion'] = np.sqrt(resultsDF['ra_motion']**2. + resultsDF['dec_motion']**2.)

     0433932   2015 04 18.27432 13 22 27.430-08 21 51.00                     W84
     0433936   2015 04 18.27756 13 22 27.460-08 21 49.60                     W84
     0433940   2015 04 18.28078 13 22 27.500-08 21 52.30                     W84
     0433945   2015 04 18.28872 13 22 27.620-08 21 51.80                     W84
     0433949   2015 04 18.29191 13 22 27.610-08 21 51.60                     W84
     0434610   2015 04 19.26575 13 22 27.680-08 21 50.10                     W84
     0434614   2015 04 19.26900 13 22 27.750-08 21 49.90                     W84
     0434618   2015 04 19.27225 13 22 27.670-08 21 54.60                     W84
     0434622   2015 04 19.27547 13 22 27.920-08 21 51.60                     W84
     0434626   2015 04 19.27871 13 22 27.690-08 21 52.20                     W84
     0435498   2015 04 21.26670 13 22 27.340-08 22 00.00                     W84
     0435502   2015 04 21.27007 13 22 27.200-08 22 04.50                     W84
     0435506   2015 04 21.27

In [4]:
# Can comment this out if you don't want to select on slow movers (total_motion is arcsec/hr.)
# Cut the dataframe based on desired brightness and speed
cutDF = resultsDF.query('v_mag > 21 and total_motion < 20')
cutDF = cutDF.reset_index(drop=True)

In [6]:
# Match the objects list to pixel values in the warps located at dataPath
dataPath = '/astro/store/epyc/users/smotherh/DECAM_Data_Reduction/Pointing_Group_300/warps/%i' % pgNum
objectDF=matchPixels(cutDF,dataPath)

In [39]:
objectDF.query('ccd==4')

,visit_id,name,ra_hour,ra_min,ra_sec,dec_deg,dec_min,dec_sec,v_mag,ra_motion,dec_motion,field,ccd,x_pixel,y_pixel,total_motion
15,433932,(231610) 2009 RK12,13,20,6.1,-7,39,37,21.4,-17,6,300,4,1533.627616,417.982628,18.027756
16,433932,2015 GK56,13,20,5.7,-7,39,9,24.3,-2,1,300,4,1427.009552,394.800850,2.236068
53,433936,(231610) 2009 RK12,13,20,6.0,-7,39,37,21.4,-17,6,300,4,1533.637379,412.322522,18.027756
54,433936,2015 GK56,13,20,5.7,-7,39,8,24.3,-2,1,300,4,1423.200271,394.781540,2.236068
91,433940,(231610) 2009 RK12,13,20,5.9,-7,39,36,21.4,-17,6,300,4,1529.838055,406.643084,18.027756
92,433940,2015 GK56,13,20,5.7,-7,39,8,24.3,-2,1,300,4,1423.200271,394.781540,2.236068
129,433945,(231610) 2009 RK12,13,20,5.7,-7,39,35,21.4,-17,6,300,4,1526.048534,395.303457,18.027756
130,433945,2015 GK56,13,20,5.6,-7,39,8,24.3,-2,1,300,4,1423.210170,389.121289,2.236068
167,433949,(231610) 2009 RK12,13,20,5.6,-7,39,34,21.4,-17,6,300,4,1522.249238,389.623945,18.027756
168,433949,2015 GK56,13,20,5.6,-7,39,8,24.3,-2,1,300,4,1423.210170,389.121289,2.236068


In [42]:
init_data = True
for i in range(62):
    full_query = res_df.query('ccd == '+str(i))
    query = full_query.drop(columns=['ra_hour','ra_min','ra_sec','dec_deg','dec_min','dec_sec','ra_motion','dec_motion'])
    if len(query) > 5:
        if init_data:
            query.to_csv('objects300.txt',sep=',')
            init_data=False
            with open('objects300.txt','a') as f:
                f.write('\n')
        else:
            query.to_csv(f,sep=',',header=False,mode='a')
            with open('objects300.txt','a') as f:
                f.write('\n')